In [ ]:
from google.oauth2.credentials import Credentials

from autogen import AssistantAgent, ConversableAgent, LLMConfig
from autogen.tools import Tool

In [ ]:
# Helper function to get Google credentials
from typing import TYPE_CHECKING, Optional, Protocol, runtime_checkable


@runtime_checkable
class GoogleCredentialsProvider(Protocol):
    def get_login_url(self) -> str: ...

    def get_credentials(self, code: str) -> Optional["Credentials"]: ...


class GoogleCredentialsLocalProvider:
    def __init__(
        client_secret_file: str,
        scopes: list[str],  # e.g. ['https://www.googleapis.com/auth/drive/readonly']
        service_name: str,  # e.g. 'sheets', 'drive', etc.
        version: str,  # e.g. 'v4'
    ) -> None:
        pass

    def get_login_url(self) -> str:
        return "http://localhost"

    def get_credentials(self, code: str) -> Optional["Credentials"]:
        pass


if TYPE_CHECKING:

    def _check_google_local_credentials_provider(x: GoogleCredentialsLocalProvider) -> GoogleCredentialsProvider:
        return x


class GoogleCredenentialsHostedProvider:
    def __init__(self, url: str, token: Optional[str] = None) -> None:
        pass

    def get_login_url(self) -> str:
        pass

    def get_credentials(self, code: str) -> Optional["Credentials"]:
        pass


if TYPE_CHECKING:

    def _check_google_hosted_credentials_provider(x: GoogleCredenentialsHostedProvider) -> GoogleCredentialsProvider:
        return x

In [ ]:
class ToolSet:
    def __init__(self, tools: list[Tool]):
        self.tools = tools

    def register_for_llm(self, agent: ConversableAgent) -> None:
        for tool in self.tools:
            tool.register_for_llm(agent)

    def register_for_execution(self, agent: ConversableAgent) -> None:
        for tool in self.tools:
            tool.register_for_execution(agent)

In [ ]:
from pathlib import Path
from typing import Annotated, Literal, Union

from pydantic import BaseModel

from autogen.tools import Depends, tool


class GoogleDriveFileInfo(BaseModel):
    name: str
    id: str
    is_folder: bool


class GoogleDriveListTool(Tool):
    def __init__(self, credentials: "Credentials") -> None:
        @tool(description="list all files in a Google Drive folder")
        def list_files_in_folder(
            folder_path: str,
            credentials: Annotated["Credentials", Depends(credentials)],
        ) -> list[GoogleDriveFileInfo]:
            pass

        super().__init__(list_files_in_folder)


class GoogleDriveToolSet(ToolSet):
    def __init__(
        self,
        *,
        credentials: "Credentials",
        download_folder: Union[Path, str],
        include: Optional[list[Literal["list_files_in_folder", "download_file", "upload_file"]]] = None,
        exclude: Optional[list[Literal["list_files_in_folder", "download_file", "upload_file"]]] = None,
    ) -> None:
        @tool(description="list all files in a Google Drive folder")
        def list_files_in_folder(
            folder_path: str,
            credentials: Annotated["Credentials", Depends(credentials)],
        ) -> list[GoogleDriveFileInfo]:
            pass

        # add download file tool
        @tool(description="download a file from Google Drive")
        def download_file(
            file_id: str,
            credentials: Annotated["Credentials", Depends(credentials)],
            destination_path: Optional[Union[Path, str]] = None,
        ) -> str:
            # return "downloaded file path"
            pass

        # add upload file tool
        @tool(description="upload a file to Google Drive")
        def upload_file(
            file_path: Union[Path, str],
            folder_id: str,
            credentials: Annotated["Credentials", Depends(credentials)],
        ) -> str:
            # return "uploaded file id"
            pass

        super().__init__([
            list_files_in_folder,
        ])


# @tool(description="list all files in a Google Drive folder")
# def list_files_in_folder(
#     folder_id: str,
#     credentials: Annotated[Credentials, Depends(credentials)],
# ) -> None:
#     pass

In [ ]:
list_tool = GoogleDriveListTool(credentials=Credentials())
download_tool = GoogleDriveDownloadTool(credentials=Credentials())

tool_set = ToolSet([list_tool, download_tool])


tool_set = GoogleDriveToolSet(credentials=Credentials(), download_folder=Path("downloads"), exclude=["upload_file"])

In [ ]:
class GoogleDriveListFilesTool(Tool):
    def __init__(self, credentials: Credentials):
        def list_files(size: int) -> list[str]:
            # service.execute(credentials, 'drive', 'v4', 'files', 'list', {'pageSize': size})
            # DA LI MOGU KORISTITI 'credentials' UNUTAR FUNKCIJE BEZ DEPENDENCY INJECITON-A?
            pass


class GoogleDriveDownloadFileTool(Tool):
    def __init__(self, credentials: Credentials, download_folder: str):
        def download_file(file_id: str) -> str:
            ...
            return "File saved to " + download_folder


class GoogleDriveUploadFileTool(Tool):
    def __init__(self, credentials: Credentials):
        def upload_file(file_path: str) -> str:
            ...
            return "File uploaded successfully"

In [ ]:
google_credentials = get_local_google_credentials(
    client_secret_file="client_secret.json",
    scopes=["https://www.googleapis.com/auth/drive"],
    service_name="drive",
    version="v3",
)

# Choose the tools you want to use
tool_set = ToolSet([
    GoogleDriveListFilesTool(google_credentials),
    GoogleDriveDownloadFileTool(google_credentials, download_folder="downloads"),
])

assistant = AssistantAgent(
    name="GoogleDriveAssistant",
    llm_config=LLMConfig(api_type="openai", model="gpt-4o-mini"),
)

In [ ]:
tool_set.register_for_llm(assistant)

assistant.run(
    message="List files",
    tools=tool_set.tools,
)